In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from bs4 import BeautifulSoup

In [2]:
data=pd.read_csv("/home/nishchay/Desktop/Reviews.csv")


In [3]:
data=data[data['Score']!=3]

data['Sentiment']=[1 if x>2 else 0 for x in data['Score']]
data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,Sentiment
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...,1
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,0
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...,1
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...,0
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...,1


In [4]:
final=data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"}, keep='first', inplace=False)

In [5]:
data.shape

(525814, 11)

In [6]:
final.shape

(364173, 11)

In [7]:
(final.size*1.0)/(data.size*1.0)*100

69.258901436629685

In [8]:
final.loc[final['HelpfulnessNumerator'] >final['HelpfulnessDenominator']]

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,Sentiment
44736,44737,B001EQ55RW,A2V0I904FH7ABY,Ram,3,2,4,1212883200,Pure cocoa taste with crunchy almonds inside,It was almost a 'love at first bite' - the per...,1
64421,64422,B000MIDROQ,A161DK06JJMCYF,"J. E. Stephens ""Jeanne""",3,1,5,1224892800,Bought This for My Son at College,My son loves spaghetti so I didn't hesitate or...,1


In [9]:
final=final[final.HelpfulnessNumerator<=final.HelpfulnessDenominator]

In [10]:
final.shape

(364171, 11)

In [11]:
(final.size*1.0)/(data.size*1.0)*100

69.258521073991943

In [12]:
final['Text'][100]

'The mouth says, "How do I love thee, let me count the ways..."<br />If you like apple products a must have item.  The only draw back, shipping cost.  These are very heavy.'

In [13]:
final['Text'][1000]

'I never in my life tasted such a good babka its crazy good! This is the real babka! That my gram mother use to make'

In [14]:
final["Text"][1500]

'You can find this 6-pack for $40 dollars less at diapers.com.  No reason they should be charging this much.'

In [15]:
final['Text'][3000]

'I am very pleased with this product. I almost did not buy them because they looked green on my computer, but when I received them they were the normal tan color that most of them are.'

In [16]:
final['Text'][9824]

'My senses were surprised by the aroma of the glistening beans as well as the satisfying flavor of the brew.<br />I substituted this bean when Amazon dropped Starbucks Sumatra from their Subscribe & Save program.<br />Now I would never go back.<br />I use a ceramic cone drip filter into a Nissan Thermal Carafe for our brew.<br />For an organic bean, this Breakfast Blend is top of the heap.<br />{{* *}}'

In [17]:
import re

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [18]:
stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

In [19]:

preprocessed_reviews = []

for sentence in final['Text'].values:
    sentence = re.sub(r"http\S+", "", sentence)
    sentence = BeautifulSoup(sentence, 'lxml').get_text()
    sentence = decontracted(sentence)
    sentence = re.sub("\S*\d\S*", "", sentence).strip()
    sentence = re.sub('[^A-Za-z]+', ' ', sentence)
    
    
    sentence = ' '.join(e.lower() for e in sentence.split() if e.lower() not in stopwords)
    preprocessed_reviews.append(sentence.strip())

In [20]:
final['Text'].values[9824]

"I don't even like coffee, but this is in a class by itself!<br /><br />There is no bitter aftertaste, and with the slightest sprinkle of chocolate creamer, it is going quickly to become my drink of choice!  I can't imagine how good it will taste with a shot of liquor in it!<br /><br />I just wish we had more! (But we will soon...thanks Amazon!)<br /><br />No matter what Jamaican coffee you buy, make sure it is certified Blue Mountain Jamaican coffee, like this one is! :)"

In [21]:
preprocessed_reviews[9824]

u'not even like coffee class no bitter aftertaste slightest sprinkle chocolate creamer going quickly become drink choice not imagine good taste shot liquor wish soon thanks amazon no matter jamaican coffee buy make sure certified blue mountain jamaican coffee like one'

In [119]:
from nltk.stem import PorterStemmer
import nltk

snob=nltk.stem.SnowballStemmer('english')
print(snob.stem('tasty'))

tasti


In [120]:
final['Cleaned Text']= preprocessed_reviews

final.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,Sentiment,Cleaned Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...,1,bought several vitality canned dog food produc...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,0,product arrived labeled jumbo salted peanuts p...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...,1,confection around centuries light pillowy citr...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...,0,looking secret ingredient robitussin believe f...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...,1,great taffy great price wide assortment yummy ...


In [121]:
# Sorting data based on time
final["Time"] = pd.to_datetime(final["Time"], unit = "s")
final = final.sort_values(by = "Time")

In [122]:
final.shape

(364171, 12)

In [123]:
X= final["Cleaned Text"]
y=final["Score"]
print("Shape of X=",X.shape)
print("Shape of y=",y.shape)

('Shape of X=', (364171,))
('Shape of y=', (364171,))


In [124]:
from sklearn.model_selection import train_test_split

X_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

print(X_train.shape, y_train.shape, x_test.shape)

((254919,), (254919,), (109252,))


In [125]:
X_train.head()

129996    buy cereal time cereal eat due numerous food a...
127570    four one half starsthis coffee packaging send ...
316605    honey maid graham crackers around years taste ...
429389    ranch dressing not sweet mussle salad keep thr...
470364    follow simple directions package easy product ...
Name: Cleaned Text, dtype: object

In [126]:
y_train.head()

129996    5
127570    4
316605    4
429389    1
470364    5
Name: Score, dtype: int64

In [ ]:
def k_classifier_brute(X_train, y_train):
    
    myList = list(range(0,50))
    neighbors = list(filter(lambda x: x % 2 != 0, myList))

    
    cv_scores = []

    
    for k in neighbors:
        knn = KNeighborsClassifier(n_neighbors=k, algorithm = "brute")
        scores = cross_val_score(knn, X_train, y_train, cv=10, scoring='accuracy')
        cv_scores.append(scores.mean())

  
    MSE = [1 - x for x in cv_scores]

    
    optimal_k = neighbors[MSE.index(min(MSE))]
    print('\nThe optimal number of neighbors is %d.' % optimal_k)

    # plot misclassification error vs k 
    plt.plot(neighbors, MSE)

    for xy in zip(neighbors, np.round(MSE,3)):
        plt.annotate('(%s, %s)' % xy, xy=xy, textcoords='data')
    plt.title("Misclassification Error vs K")
    plt.xlabel('Number of Neighbors K')
    plt.ylabel('Misclassification Error')
    plt.show()

    print("the misclassification error for each k value is : ", np.round(MSE,3))
    return optimal_k

In [127]:
from sklearn.feature_extraction.text import CountVectorizer 

bow = CountVectorizer()
X_train = bow.fit_transform(X_train)
x_test= bow.transform(x_test)

test=bow.transform(["This is a great phone"," This is a disasterous phone"])
print(X_train,x_test)

(<254919x97201 sparse matrix of type '<type 'numpy.int64'>'
	with 8382364 stored elements in Compressed Sparse Row format>, <109252x97201 sparse matrix of type '<type 'numpy.int64'>'
	with 3567723 stored elements in Compressed Sparse Row format>)


In [128]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from collections import Counter
from sklearn.metrics import accuracy_score
from sklearn import model_selection


knn= KNeighborsClassifier(n_neighbors=5)


knn.fit(X_train, y_train)
pred = knn.predict(x_test)

train_acc= (knn.score(x_test,pred))*100
train_error= (100-train_acc)
acc_bow = accuracy_score(y_test, pred) * 100
test_error= (100- acc_bow)

print("Training Error is ", train_error)
print("Test Error is ",test_error)
print("Training Accuracy is", train_acc)
print("The accuracy of the knn classifier %", (acc_bow))

print(knn.predict(test))

('Training Error is ', 0.0)
('Test Error is ', 35.471204188481678)
('Training Accuracy is', 100.0)
('The accuracy of the knn classifier %', 64.528795811518322)
[1 5]
